## FBA simulation toy_atp
Simulating the FBA part.

In [1]:
from __future__ import print_function, absolute_import

import pandas as pd
import cobra
from matplotlib import pyplot as plt

from sbmlutils import fbc
from sbmlutils.dfba.analysis import set_matplotlib_parameters
from sbmlutils.dfba.toy_atp import model_factory

set_matplotlib_parameters()

In [2]:
# load model
sbml_path = './results/v{}/toy_atp_fba.xml'.format(model_factory.version)
print(sbml_path)
model = cobra.io.read_sbml_model(sbml_path)
cobra.io.sbml3.validate_sbml_model(sbml_path)

./results/v2/toy_atp_fba.xml


(<Model toy_atp_fba at 0x7f13ea2d79e8>,
 {'SBML errors': [], 'other': [], 'validator': [], 'warnings': []})

In [3]:
# objective function & boundaries
# pprint(mfba.objective)
df = fbc.cobra_reaction_info(model)
print(df)
print("reactions:", len(model.reactions))
print("metabolites:", len(model.metabolites))
print("genes:", len(model.genes))

          lb     ub reversibility boundary objective_coefficient  \
R1         0   1000         False    False                     0   
R2         0   1000         False    False                     0   
R3         0  0.001         False    False                     1   
EX_atp -1000   1000          True     True                     0   
EX_adp -1000   1000          True     True                     0   
EX_glc -1000   1000          True     True                     0   
EX_pyr -1000   1000          True     True                     0   

             forward_variable                     reverse_variable  
R1        0.0 <= R1 <= 1000.0           0 <= R1_reverse_cda52 <= 0  
R2        0.0 <= R2 <= 1000.0           0 <= R2_reverse_8c6d2 <= 0  
R3         0.0 <= R3 <= 0.001           0 <= R3_reverse_5c108 <= 0  
EX_atp  0 <= EX_atp <= 1000.0  0 <= EX_atp_reverse_dfd97 <= 1000.0  
EX_adp  0 <= EX_adp <= 1000.0  0 <= EX_adp_reverse_76b83 <= 1000.0  
EX_glc  0 <= EX_glc <= 1000.0  0 <= EX_gl

In [4]:
# Exchange reactions
ex_idx = df.index.str.contains('^EX_')
df[ex_idx]

,lb,ub,reversibility,boundary,objective_coefficient,forward_variable,reverse_variable
EX_atp,-1000,1000,True,True,0,0 <= EX_atp <= 1000.0,0 <= EX_atp_reverse_dfd97 <= 1000.0
EX_adp,-1000,1000,True,True,0,0 <= EX_adp <= 1000.0,0 <= EX_adp_reverse_76b83 <= 1000.0
EX_glc,-1000,1000,True,True,0,0 <= EX_glc <= 1000.0,0 <= EX_glc_reverse_8e37d <= 1000.0
EX_pyr,-1000,1000,True,True,0,0 <= EX_pyr <= 1000.0,0 <= EX_pyr_reverse_03be5 <= 1000.0


In [5]:
# optimize
s = model.optimize(objective_sense="maximize")
model.summary(fva=True)

IN FLUXES                       OUT FLUXES                    OBJECTIVES
------------------------------  ----------------------------  ------------
id      Flux  Range             id      Flux  Range           R3  0.001
----  ------  ----------------  ----  ------  --------------
glc   0.0005  [0.0005, 0.0005]  pyr    0.001  [0.001, 0.001]


In [6]:
# pfba (minimal flux)
# no difference, the flux variability analysis 
# already showed us that the model has unique solution under given bounds 
s = model.optimize(objective_sense="maximize")
cobra.flux_analysis.pfba(model)
model.summary(fva=True)

IN FLUXES                       OUT FLUXES                    OBJECTIVES
------------------------------  ----------------------------  ------------
id      Flux  Range             id      Flux  Range           R3  0.001
----  ------  ----------------  ----  ------  --------------
glc   0.0005  [0.0005, 0.0005]  pyr    0.001  [0.001, 0.001]


In [9]:
s.fluxes

R1        0.0005
R2        0.0005
R3        0.0010
EX_atp    0.0000
EX_adp    0.0000
EX_glc   -0.0005
EX_pyr    0.0010
Name: fluxes, dtype: float64